# Advanced Examples

https://pythonhosted.org/mcerp/advanced_example.html

In [1]:
from mcerp3 import *  # N, U, Gamma, Beta, correlate, etc.

In this section, some more advanced/complex usages will be presented.

## Volumetric Gas Flow Through an Orifice Meter

Here’s an interesting engineering example, showing how the random effects of input parameters propagates through the calculation of the volumetric gas flow through an orifice meter:

In [2]:
import mcerp3.umath as umath  # sin, exp, sqrt, etc.

H = N(64, 0.5)
M = N(16, 0.1)
P = N(361, 2)
t = N(165, 0.5)
C = 38.4
Q = C*umath.sqrt((520*H*P)/(M*(t + 460)))
Q.describe()

MCERP Uncertain Value:
 > Mean...................  1331.0002189394509
 > Variance...............  57.79636117924653
 > Skewness Coefficient...  0.010971840098889973
 > Kurtosis Coefficient...  3.0424825890360565



Interestingly enough, even though the calculations involve multiplication, division, and a square-root, the result appears to be very close to a Normal distribution (Q ~ N(1331, 7.6)).

## Manufacturing Tolerance Stackup

In a certain manufacturing plant, a record of process data is taken and just happens to fit a Gamma distribution with a mean of 1.5 and a variance of 0.25. With an assembly of three of these parts, our analysis for tolerance stackup would proceed as follows. We first need to convert the distribution statistics to the shape parameters k and theta, which are found by the relations:

In [3]:
mean = 1.5
variance = 0.25
k = mean**2/variance
theta = variance/mean
k, theta

(9.0, 0.16666666666666666)

We can now proceed with the analysis:

In [4]:
x = Gamma(k, theta)
y = Gamma(k, theta)
z = Gamma(k, theta)
w = x + y + z
w.describe()

MCERP Uncertain Value:
 > Mean...................  4.499990905005162
 > Variance...............  0.7307952293492459
 > Skewness Coefficient...  0.36306901636440037
 > Kurtosis Coefficient...  3.195756126887414



Due to the skewed nature of the inputs, the output is also slightly skewed.

## Scheduling Facilities

At a manufacturing plant, that has six test-and-repair stations, data is collected about the amount of time that product is present at each station:

1. Station 1: Normally distributed, mean of 10 hours, variance of 1 hour.

2. Station 2: Normally distributed, mean of 20 hours, variance of 2 hours.

3. Station 3: Gamma distributed, mean of 1.5 hours, variance of 0.25 hours.

4. Station 4: Gamma distributed, mean of 10 hours, variance of 10 hours.

5. Station 5: Exponentially distributed, mean of 0.2 hours, variance of 0.04 hours (decay constant lamda=5).

6. Station 6: Chi-squared distributed, mean of 10 hours, variance of 20 hours (degree of freedom v=10).

Management wants to understand the uncertainty associated with the whole process:

In [5]:
# Station 1
s1 = N(10, 1)

# Station 2
s2 = N(20, 2**0.5)

# Station 3
mn1 = 1.5
vr1 = 0.25
k1 = mn1**2/vr1
theta1 = vr1/mn1
s3 = Gamma(k1, theta1)

# Station 4
mn2 = 10
vr2 = 10
k2 = mn2**2/vr2
theta2 = vr2/mn2
s4 = Gamma(k2, theta2)

# Station 5
s5 = Exp(5)

# Station 6
s6 = Chi2(10)

T = s1 + s2 + s3 + s4 + s5 + s6
T.describe()

MCERP Uncertain Value:
 > Mean...................  51.69993939943796
 > Variance...............  33.45313734612313
 > Skewness Coefficient...  0.5031370598016396
 > Kurtosis Coefficient...  3.4167807899238007



From this analysis, we see that the average process time is about 51.7 hours, but the variance is quite large (standard deviation = 5.8 hours). This gives management the desire to understand which is the greatest contributors, so we can analyze the standard deviations of each process step:

In [6]:
for i, si in enumerate([s1, s2, s3, s4, s5, s6]):
    print('Station', i + 1, ':', si.std)

Station 1 : 0.9998378884429786
Station 2 : 1.4139759655466593
Station 3 : 0.4999115035264646
Station 4 : 3.1615715602197834
Station 5 : 0.1998974167020453
Station 6 : 4.472515382537669


This would seem to indicate that management could focus their efforts on making the cycle times of stations 4 and 6 more consistent.

It may also be useful to understand the probability that a complete cycle will exceed a certain amount, say at 59, 62 and 68 hours:

In [7]:
prob = [T > hr for hr in [59, 62, 68]]
prob

[0.10880000000000001, 0.05130000000000001, 0.008000000000000007]

That is to say that it is expected that the entire process will take 59 hours approximately 11% of the time, 62 hours 5% of the time, and 68 hours about 1% of the time.